In [ ]:
# 读取文件 并将地址存放再vins[]中
import pandas as pd
import os
inf = pd.read_excel('/home/mw/input/contest_data5759/dataset_xgame_battery/车型信息.xlsx')
test = pd.read_csv('/home/mw/input/contest_data5759/dataset_xgame_battery/vmodelseq=123115361/vin=01e6453a3fc9d51aafee11e8245dcaf5/part-00092-1622b5d7-b0e7-4a3c-8bb0-5207ae82594a.c000.csv')
test.info()

files = '/home/mw/input/contest_data5759/dataset_xgame_battery'

# Creat list of model
model = os.listdir(files)
model.remove('车型信息.xlsx')
print(model)

# Creat list of path to each csv of car
vins = []
for m in model:
    vin = os.listdir(os.path.join(files,m))
    for v in vin:
        csv_name = os.listdir(os.path.join(files,m,v))[0]
        vins.append(os.path.join(files,m,v,csv_name))

In [ ]:
# Set the environment for data visualization(seaborn)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Seaborn setup Complete")

In [ ]:
# 片段提取

print(f'The total num of samples:{len(vins)}') 

import numpy as np

# enable them when initial the search
# period 存储行驶、充电的区间 （vins, run start/run end, battery start/batterty end）
# period = np.zeros((len(vins),5,10000))
# # err存储每种特征出现异常值的车型
# err = {}
# for i in range(len(pd.read_csv(vins[1]).columns)):
#     err[i] = []

def Preprocess(vin_file,vin_idx):
    df = pd.read_csv(vin_file)
    
    # 整理行驶、充电的区间，阈值为10 0000 ms
    run_idx, charge_idx = 1,1
    period[vin_idx,1,0],period[vin_idx,2,0] = df.at[0, 'collectiontime'],df.at[0, 'collectiontime']
    run, charge = False, False
    for idx in range(len(df)):
        if (df.at[idx, 'vehicledata_vehiclestatus']== 1):
            # 如果是周期的起始点
            if (not run) & ((df.at[idx, 'collectiontime'] - period[vin_idx,2,run_idx-1])>10000):
                period[vin_idx,1,run_idx] = df.at[idx, 'collectiontime']
                run = True
                # print(run_idx)
                # print(df.at[idx, 'collectiontime'])
            else:
                period[vin_idx,2,run_idx] = df.at[idx, 'collectiontime']
                try:
                    if(df.at[idx+1, 'vehicledata_vehiclestatus']!= 1):
                        run = False
                        run_idx = run_idx+1
                except:
                    pass
                    continue

        if (df.at[idx, 'vehicledata_chargestatus']== 1):
            # 如果是周期的起始点
            if (not charge) & ((df.at[idx, 'collectiontime'] - period[vin_idx,2,charge_idx-1])>10000):
                period[vin_idx,3,charge_idx] = df.at[idx, 'collectiontime']
                charge = True
                # print(run_idx)
                # print(df.at[idx, 'collectiontime'])
            else:
                period[vin_idx,4,charge_idx] = df.at[idx, 'collectiontime']
                try:
                    if(df.at[idx+1, 'vehicledata_chargestatus']!= 1):
                        charge = False
                        charge_idx = charge_idx+1
                except:
                    pass
                    continue
        

        # 异常值处理
        for i in [1,2,8,9,14,15,16,17,18,19,20,22,23,24,25,26,27]:
            if df.iloc[idx,i] > 253 :
                if vin_idx not in err[i]:
                    err[i].append(vin_idx)
        for i in [4,6,7,18,21]:
            if df.iloc[idx,i] > 60000 :
                if vin_idx not in err[i]:
                    err[i].append(vin_idx)
        if df.loc[idx,	'vehicledata_summileage'] > 10000000 :
            if vin_idx not in err[5]:
                    err[5].append(vin_idx)
    print(vin_file)
    print(vin_idx)        
    print(run_idx)       
    print(charge_idx)
    #print(err)
    # save the err and period
    np.save('/home/mw/project/period.npy',period)
    np.save('/home/mw/project/err.npy',err)


# load and precess
period = np.load('/home/mw/project/period.npy',allow_pickle=True)
err = np.load('/home/mw/project/err.npy',allow_pickle=True).item()
idx = 197
period[idx][:][:]= 0
# this is big task, enable it only when in need
for t in range(idx,len(vins)):
    Preprocess(vins[t],t)

In [ ]:
# 绘制1 2 3 4 不同充电状态下内阻数据的分布图
def plot_r(df_r,feature,c_status=None,soc_min=85,soc_max=85,feature2=None,hue='vehicledata_chargestatus',time_select=None,peak_plot=True): 

    c1 = df_r[df_r['vehicledata_soc'] >= soc_min]
    c1 = c1[c1['vehicledata_soc'] <= soc_max]
    c1 = c1[c1['vehicledata_sumvoltage']<500]
    c1 = c1[c1['vehicledata_insulationresistance']>2000]
    c1 = c1[c1['vehicledata_insulationresistance']<6000]


    plt.figure(figsize=(16,4))
    if time_select != None:
        select = (c1['collectiontime'].max() - c1['collectiontime'].min())/100
        select = int(select)
        start,end = (c1['collectiontime'].min() + time_select[0]*select),(c1['collectiontime'].min() + time_select[1]*select)
        c1 = c1[c1['collectiontime']>start]
        c1 = c1[c1['collectiontime']<end]
    if c_status != None:
        c1 = c1[c1['vehicledata_chargestatus']==c_status]
    if feature2 != None:
        sns.scatterplot(data=c1,x=feature,y=feature2,hue=hue)
        return 0
    sns.scatterplot(data=c1,x='collectiontime',y=feature,hue=hue,size=0.01)

 # 寻找峰值
def get_peak(df_r,c_status=None,soc_min=100,soc_max=100,time_select=None):
    c1 = df_r[df_r['vehicledata_soc'] >= soc_min]
    c1 = c1[c1['vehicledata_soc'] <= soc_max]
    c1 = c1[c1['vehicledata_sumvoltage']<500]
    # print(c1)
    if c_status != None:
        c1 = c1[c1['vehicledata_chargestatus']==c_status]
    if time_select != None:
        select = (c1['collectiontime'].max() - c1['collectiontime'].min())/100
        select = int(select)
        start,end = (c1['collectiontime'].min() + time_select[0]*select),(c1['collectiontime'].min() + time_select[1]*select)
        c1 = c1[c1['collectiontime']>start]
        c1 = c1[c1['collectiontime']<end]
    
    plt.figure(figsize=(16,4))
    c1 = c1['vehicledata_sumvoltage'].to_list()
    # print(c1)
    if len(c1) < 40 :
        peaks, _ = find_peaks(c1, height=0)
    else:
        peaks, _ = find_peaks(c1, height=0, distance=(len(c1)/10))
    sns.scatterplot(x=range(len(c1)),y=c1,size=0.01)
    y_peaks = []
    for i in range(len(peaks)):
        y_peaks.append(c1[peaks[i]])
    sns.lineplot(x=peaks,y=y_peaks,marker='+')

In [ ]:
get_peak(pd.read_csv(vins[146]),time_select=[0,100])

![Alt text](image.png)

In [ ]:
v_temp = pd.read_csv(vins[123])
v_temp = v_temp[v_temp['vehicledata_speed']!=0]
v_temp = v_temp[v_temp['vehicledata_speed']<300]
plot_r(v_temp,'vehicledata_sumvoltage',None,89,90,feature2='vehicledata_speed',hue='collectiontime')



![Alt text](image-1.png)

In [ ]:
# 此图不要动
plot_r(pd.read_csv(vins[146]),'vehicledata_sumvoltage')
plot_r(pd.read_csv(vins[146]),'extremevalue_maxbatterysinglevoltageval')
plot_r(pd.read_csv(vins[146]),'vehicledata_soc',3,60,100,feature2='vehicledata_sumvoltage',hue='collectiontime')

![Alt text](image-2.png)

In [ ]:
# 这是另一个（稍微新一些的）函数封装
# 读取文件 
import pandas as pd
import multiprocessing as MP
import numpy as np
import os
import sys
from scipy.signal import find_peaks
# Set the environment for data visualization(seaborn)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Seaborn setup Complete")

inf = pd.read_excel('/home/mw/input/contest_data5759/dataset_xgame_battery/车型信息.xlsx')
test = pd.read_csv('/home/mw/input/contest_data5759/dataset_xgame_battery/vmodelseq=123115361/vin=01e6453a3fc9d51aafee11e8245dcaf5/part-00092-1622b5d7-b0e7-4a3c-8bb0-5207ae82594a.c000.csv')
# test.info()

files = '/home/mw/input/contest_data5759/dataset_xgame_battery'

# Creat list of model
model = os.listdir(files)
model.remove('车型信息.xlsx')
print(model)

# Creat list of path to each csv of car
vins = []
for m in model:
    vin = os.listdir(os.path.join(files,m))
    for v in vin:
        csv_name = os.listdir(os.path.join(files,m,v))[0]
        vins.append(os.path.join(files,m,v,csv_name))
print(len(vins))

# def some func
def plot_r(df_r,feature,c_status=None,soc_min=90,soc_max=100,feature2=None,hue='vehicledata_chargestatus'): 

    c1 = df_r[df_r['vehicledata_soc'] >= soc_min]
    c1 = c1[c1['vehicledata_soc'] <= soc_max]
    c1 = c1[c1['vehicledata_sumvoltage']<500]

    plt.figure(figsize=(16,4))
    if c_status != None:
        c1 = c1[c1['vehicledata_chargestatus']==c_status]
    if feature2 != None:
        sns.scatterplot(data=c1,x=feature,y=feature2,hue=hue,linewidth=0.1)
        return 0
    sns.scatterplot(data=c1,x='collectiontime',y=feature,hue=hue,linewidth=0.1)

def get_mean(df,start,end):
    # limit time 
    df = df[df['vehicledata_sumvoltage']<500]
    df = df[df['vehicledata_chargestatus']==4]
    df = df[df['collectiontime']>=start]
    df = df[df['collectiontime']<=end]
    df = df[df['vehicledata_soc']==100]
    return df['vehicledata_sumvoltage'].mean()

# 现在通过峰值选取后的方式计算每组的性能
 # 寻找峰值
def check_bad(idx_list,cklist):
    # 将3sigma的之外的数据扔掉
    if cklist == []:
        return [],[]
    mean = np.mean(cklist)
    std = np.std(cklist)
    isout=[False if (t-mean)*(t-mean) > 5*std*std else True for t in cklist]
    return [cki for cki,ckii in zip(idx_list,isout) if ckii], [ckj for ckj,ckjj in zip(cklist,isout) if ckjj]
# print(check_bad([1,1,1,1,1,1,1,1,1,1,1],[1000,1000,1000,10,1000,1000,1000,1000,1000]))
def get_peak(df_r,c_status=None,soc_min=100,soc_max=100,time_select=None,isplot=False,check=True):
    
    if time_select != None:
        select = (df_r['collectiontime'].max() - df_r['collectiontime'].min())/100
        select = int(select)
        start,end = (df_r['collectiontime'].min() + time_select[0]*select),(df_r['collectiontime'].min() + time_select[1]*select)
        df_r = df_r[df_r['collectiontime']>start]
        df_r = df_r[df_r['collectiontime']<end]
    df_r = df_r[df_r['vehicledata_soc'] >= soc_min]
    df_r = df_r[df_r['vehicledata_soc'] <= soc_max]
    df_r = df_r[df_r['vehicledata_sumvoltage']<500]
    
    

    df_r = df_r['vehicledata_sumvoltage'].to_list()
    if len(df_r) < 20 :
        peaks, _ = find_peaks(df_r, height=0)
    elif len(df_r) < 400:
        peaks, _ = find_peaks(df_r, height=0, distance=(len(df_r)/10))
    else:
        peaks, _ = find_peaks(df_r, height=0, distance=(len(df_r)/50))
    
    y_peaks = []
    for i in range(len(peaks)):
        y_peaks.append(df_r[peaks[i]])
    if check:
        peaks,y_peaks = check_bad(peaks,y_peaks )
    if isplot:
        plt.figure(figsize=(16,4))
        sns.scatterplot(x=range(len(df_r)),y=df_r,size=0.01)
        sns.lineplot(x=peaks,y=y_peaks,marker='+')
    
    # peaks是峰值处索引值，y_peaks是峰值
    return peaks,y_peaks

    
def epoch_peak(list1,save_path,isprint=False,c_status=None,soc_min=100,soc_max=100,time_select=[0,5]):
    # 得到一个epoch 的peak 
    # list1: index list of assigned epoch 


    # list1 = np.load('list1.npy',allow_pickle=True)
    list_peak = [] # 存储每一个健康区段的峰值，用于后面求均值
    print(save_path)
    for i in list1:
        try:
            peaks,y_peaks = get_peak(pd.read_csv(vins[i])[['collectiontime','vehicledata_sumvoltage','vehicledata_soc','vehicledata_chargestatus']],c_status=c_status,soc_min=soc_min,soc_max=soc_max,time_select=time_select)          
            # 去掉 y_peaks的中的nan
            try:
                if isprint:
                    print(i,y_peaks)
                y_peaks = [t for t in y_peaks if not np.isnan(t)]
            except:
                pass
            # fix this later
            if y_peaks == []:
                continue
            list_peak.append(np.mean(y_peaks))
        except:
            print(f'vin : {i} fail')
            pass
        np.save(save_path,list_peak)
    print(list_peak)
    # 消除nan
    try:
        list_peak= [t for t in list_peak if not np.isnan(t)]
    except:
        pass
    print(f'The mean value of {save_path} peak:',np.mean(list_peak))
    np.save(save_path,list_peak)
    return np.mean(list_peak)

In [ ]:
# 这是在HI计算中采用的封装
# 读取文件 
import pandas as pd
import multiprocessing as MP
import numpy as np
import os
import sys
from scipy.signal import find_peaks
# Set the environment for data visualization(seaborn)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Seaborn setup Complete")

inf = pd.read_excel('/home/mw/input/contest_data5759/dataset_xgame_battery/车型信息.xlsx')
test = pd.read_csv('/home/mw/input/contest_data5759/dataset_xgame_battery/vmodelseq=123115361/vin=01e6453a3fc9d51aafee11e8245dcaf5/part-00092-1622b5d7-b0e7-4a3c-8bb0-5207ae82594a.c000.csv')
# test.info()

files = '/home/mw/input/contest_data5759/dataset_xgame_battery'

# Creat list of model
model = os.listdir(files)
model.remove('车型信息.xlsx')
print(model)

# Creat list of path to each csv of car
vins = []
for m in model:
    vin = os.listdir(os.path.join(files,m))
    for v in vin:
        csv_name = os.listdir(os.path.join(files,m,v))[0]
        vins.append(os.path.join(files,m,v,csv_name))
print(len(vins))

# def some func
def plot_r(df_r,feature,c_status=None,soc_min=90,soc_max=100,feature2=None,hue='vehicledata_chargestatus'): 

    c1 = df_r[df_r['vehicledata_soc'] >= soc_min]
    c1 = c1[c1['vehicledata_soc'] <= soc_max]
    c1 = c1[c1['vehicledata_sumvoltage']<500]

    plt.figure(figsize=(16,4))
    if c_status != None:
        c1 = c1[c1['vehicledata_chargestatus']==c_status]
    if feature2 != None:
        sns.scatterplot(data=c1,x=feature,y=feature2,hue=hue,linewidth=0.1)
        return 0
    sns.scatterplot(data=c1,x='collectiontime',y=feature,hue=hue,linewidth=0.1)

def get_mean(df,start,end):
    # limit time 
    df = df[df['vehicledata_sumvoltage']<500]
    df = df[df['vehicledata_chargestatus']==4]
    df = df[df['collectiontime']>=start]
    df = df[df['collectiontime']<=end]
    df = df[df['vehicledata_soc']==100]
    return df['vehicledata_sumvoltage'].mean()

# 现在通过峰值选取后的方式计算每组的性能
 # 寻找峰值
def check_bad(idx_list,cklist):
    # 将3sigma的之外的数据扔掉
    if cklist == []:
        return [],[]
    mean = np.mean(cklist)
    std = np.std(cklist)
    isout=[False if (t-mean)*(t-mean) > 5*std*std else True for t in cklist]
    return [cki for cki,ckii in zip(idx_list,isout) if ckii], [ckj for ckj,ckjj in zip(cklist,isout) if ckjj]
# print(check_bad([1,1,1,1,1,1,1,1,1,1,1],[1000,1000,1000,10,1000,1000,1000,1000,1000]))
def get_peak(df_r,c_status=None,soc_min=100,soc_max=100,time_select=None,isplot=False,check=True):
    
    if time_select != None:
        select = (df_r['collectiontime'].max() - df_r['collectiontime'].min())/100
        select = int(select)
        start,end = (df_r['collectiontime'].min() + time_select[0]*select),(df_r['collectiontime'].min() + time_select[1]*select)
        df_r = df_r[df_r['collectiontime']>start]
        df_r = df_r[df_r['collectiontime']<end]
    df_r = df_r[df_r['vehicledata_soc'] >= soc_min]
    df_r = df_r[df_r['vehicledata_soc'] <= soc_max]
    df_r = df_r[df_r['vehicledata_sumvoltage']<500]
    
    

    df_r = df_r['vehicledata_sumvoltage'].to_list()
    if len(df_r) < 20 :
        peaks, _ = find_peaks(df_r, height=0)
    else:
        peaks, _ = find_peaks(df_r, height=0, distance=(len(df_r)/10))
    # peaks容易跑不出来，虽然在采集大规模数据时并不影响，但在针对个体时容易丢掉所有数据，如果没有peaks就返回原数据
    if len(peaks) == 0:
        peaks = list(range(len(df_r)))
    
    y_peaks = []
    for i in range(len(peaks)):
        y_peaks.append(df_r[peaks[i]])
    if check:
        peaks,y_peaks = check_bad(peaks,y_peaks )
    if isplot:
        plt.figure(figsize=(16,4))
        sns.scatterplot(x=range(len(df_r)),y=df_r,size=0.01)
        sns.lineplot(x=peaks,y=y_peaks,marker='+')
    
    # peaks是峰值处索引值，y_peaks是峰值
    # print(y_peaks)
    return peaks,y_peaks

    
def epoch_peak(list1,save_path,isprint=False,c_status=None,soc_min=100,soc_max=100,time_select=[0,5]):
    # 得到一个epoch 的peak 
    # list1: index list of assigned epoch 


    # list1 = np.load('list1.npy',allow_pickle=True)
    list_peak = [] # 存储每一个健康区段的峰值，用于后面求均值
    print(save_path)
    for i in list1:
        try:
            peaks,y_peaks = get_peak(pd.read_csv(vins[i])[['collectiontime','vehicledata_sumvoltage','vehicledata_soc','vehicledata_chargestatus']],c_status=c_status,soc_min=soc_min,soc_max=soc_max,time_select=time_select)          
            # 去掉 y_peaks的中的nan
            try:
                if isprint:
                    print(i,y_peaks)
                y_peaks = [t for t in y_peaks if not np.isnan(t)]
            except:
                pass
            # fix this later
            if y_peaks == []:
                continue
            list_peak.append(np.mean(y_peaks))
        except:
            print(f'vin : {i} fail')
            pass
        np.save(save_path,list_peak)
    print(list_peak)
    # 消除nan
    try:
        list_peak= [t for t in list_peak if not np.isnan(t)]
    except:
        pass
    print(f'The mean value of {save_path} peak:',np.mean(list_peak))
    np.save(save_path,list_peak)
    return np.mean(list_peak)

In [ ]:
# 以下皆为HI计算（先确定基准）

# 返回前10%
def take(a,interval=1):
    '''
    返回list前10%的数据
    interval : take的区间
        1 : 0-10%
        2 : 10-20%
    '''
    a = np.sort(a)
    if int(len(a)/10) > 0 :
        keep = int(len(a)/10)
    else:
        print('no enough value!')
        return a

    a = a[(interval-1)*keep:interval*keep]
    return a

In [ ]:
# 得到最健康状态
soc_idx=[20,40,60,80,100]

for ct in range(1,11):
    # std :[] 存了五个值，分别对应在一定健康等级下 不同soc对应的电压基准值
    std=[]
    for i in soc_idx:
        path = os.path.join('/home/mw/project/soc_u_data',f'health_soc_{i}','soc_u_list_0_5.npy')
        a = take(np.load(path),ct).mean()
        std.append(a)
    print(std)

![Alt text](image-3.png)

In [ ]:
# 生命周期 90-95%
# 最差情况
soc_idx=[20,40,60,80,100]

for ct in range(1,11):
    # std :[] 存了五个值，分别对应在一定健康等级下 不同soc对应的电压基准值
    std=[]
    for i in soc_idx:
        path = os.path.join('/home/mw/project/soc_u_data',f'health_soc_{i}','soc_u_list_90_95.npy')
        a = take(np.load(path),ct).mean()
        std.append(a)
    print(std)

![Alt text](image-4.png)

In [ ]:
# 跑一辆车的自对比实验（不同生命周期）
# 单辆车得到健康状态
# 如果有某项为nan的话，用中值代替
def get_hea(vin_idx,t):
    soc=[20,40,60,80,100]
    status=[]
    for i in soc:
        peak,ypeaks = get_peak(pd.read_csv(vins[vin_idx])[['collectiontime','vehicledata_sumvoltage','vehicledata_soc','vehicledata_chargestatus']],soc_min=i,soc_max=i,time_select=[t,t+10])
        if ypeaks == []:
            status.append(0.5*(std_good[int(i/20)-1]+std_bad[int(i/20)-1)])
            continue
        status.append(np.mean(ypeaks)-std_bad[int(i/20)-1])
    listi = [status[i]/(std_good[i]-std_bad[i]) for i in range(5)]
    print(listi)
    return listi
hea = []    
for i in range(10): 
    l1 = get_hea(146,i)
    hea.append(0.05*(l1[0]+l1[1]+l1[2]+l1[3])+0.8*l1[4])
print(hea)